# Detecting Distributed Denial of Service (DDoS) Attacks using Machine Learning

This notebook investigates the use of various Python-based machine learning and data science libraries to build a model capable of detecting DDoS attacks in network traffic. The goal is to classify network flows as either benign or DDoS attack traffic based on extracted flow-level features.

We use  CICDDoS2019 dataset and for upcoming work we will combine CICDDOS2019 and  UL-ECE-DDoS-H-IoT-Datasets2025, allowing the model to learn both traditional enterprise network behavior and IoT healthcare network attack patterns.

# We’re going to take the following approach:

* Problem Definition

* Data

* Evaluation

* Features

* Modelling

* Experimentation

# 1. Problem Definition

`Given flow-based network traffic features, can we predict whether a network flow corresponds to a DDoS attack or benign traffic?`

# 2. Data

The data used in this project comes from two publicly available datasets:

1 : CICDDoS2019 Dataset

    Provided by the Canadian Institute for Cybersecurity (CIC)
    
    Contains realistic DDoS attack traffic and benign traffic captured in a controlled environment
    
    Includes multiple DDoS attack types such as SYN flood, UDP flood, DNS, LDAP, MSSQL, SSDP, and others
    
    Flow features are extracted using CICFlowMeter

2: UL-ECE-DDoS-H-IoT-Datasets2025

    Designed for healthcare IoT environments
    
    Contains labeled benign and DDoS traffic over protocols such as MQTT and UDP
    
    Represents modern IoT-based DDoS attack behavior

These two datasets are preprocessed and combined by selecting common numerical flow features and standardizing labels into a binary format.

# 3. Evaluation

If we can achieve high detection accuracy (≥ 95%), along with strong precision, recall, and F1-score, during the proof-of-concept phase, the project will be considered successful and suitable for further research and deployment.

Accuracy alone is not sufficient; therefore, false positives and false negatives are carefully analyzed due to their importance in real-world intrusion detection systems.

# 4. Features

The datasets consist of flow-based network features that describe statistical properties of packet flows between source and destination hosts.

    Sample Data Dictionary (Common Flow Features)
    
    Flow Duration – Total time duration of the flow
    
    Total Fwd Packets – Number of packets sent from source to destination
    
    Total Bwd Packets – Number of packets sent from destination to source
    
    Total Length of Fwd Packets – Total bytes sent forward
    
    Total Length of Bwd Packets – Total bytes sent backward
    
    Flow Bytes/s – Rate of bytes transmitted per second
    
    Flow Packets/s – Rate of packets transmitted per second
    
    Packet Length Mean – Average packet size in the flow
    
    Packet Length Variance – Variability in packet sizes
    
    Fwd Packet Length Mean – Mean packet length in forward direction
    
    Bwd Packet Length Mean – Mean packet length in backward direction
    
    SYN Flag Count – Number of SYN packets (important for SYN flood detection)
    
    ACK Flag Count – Number of ACK packets
    
    PSH / URG Flag Count – Indicators of abnormal traffic behavior
    
    Down/Up Ratio – Ratio of download to upload packets

Target Variable

    `Class`

        Benign
    
        Attack

These features help distinguish normal network behavior from high-volume, abnormal traffic patterns typically generated during DDoS attacks.

# 5. Modelling

    Several machine learning models are trained and evaluated, including:

    Logistic Regression
    
    Random Forest
    
    XGBoost
    
    Support Vector Machine (SVM)
    
    Neural Networks (MLP / LSTM for temporal behavior)
    
    Models are trained on the combined dataset to improve generalization across different network environments.

# 6. Experimentation

    Experiments include:

    Training on the combined dataset
    
    Cross-dataset validation (train on CICDDoS2019, test on IoT dataset)
    
    Feature importance analysis
    
    Performance comparison between classical ML and deep learning models

In [6]:
# Importing Libraries For EDA and Plotting 
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

# Importing Models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

# Model Evaluation 
from sklearn.metrics import f1_score,precision_score,recall_score
from sklearn.metrics import confusion_matrix, classification_report 
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn.metrics